In [3]:
import jax.numpy as np
import numpy as onp
from functools import partial
from jax import vmap
from jax.lax import scan
from jax.lax import cond
from jax import random
from jax import jit
from jax import jacrev
from jax.lax import stop_gradient
import matplotlib.pylab as plt
import matplotlib as mpl
from sklearn import manifold, datasets
import seaborn as sns
import jax

#from jax.config import config
#config.update("jax_debug_nans", True)

import jax.numpy as np
from utils import MidpointNormalize, load_data
from jax import random, flatten_util, vjp, jvp, custom_vjp, jacfwd, jacrev, vmap, grad
import matplotlib.pylab as plt
import seaborn as sns
from tsne_jax import *
import time

In [4]:
from sklearn.datasets import make_blobs, make_classification
#X, y = load_data(40)
X, y = make_blobs(n_samples=50, n_features=1000, centers=4, random_state=0, shuffle=False, cluster_std=[0.1, 3, 3, 3])
#X, y = make_classification(n_classes=5, n_samples=50, n_features=1000, random_state=42)
key = random.PRNGKey(41)
#X = onp.array(random.normal(key, shape=(50, 50)))
y_guess = random.normal(key, shape=(X.shape[0], 2))
#Y_star = TSNE(n_components=2, learning_rate=200, init=onp.array(y_guess), perplexity=30).fit_transform(X)
Y_star = tsne_fwd(X, y_guess)

X_flat, X_unflattener = flatten_util.ravel_pytree(X)   # row-wise
Y_flat, Y_unflattener = flatten_util.ravel_pytree(Y_star) 

===> Finding 49 nearest neighbors using Annoy approximate search using euclidean distance...
   --> Time elapsed: 0.02 seconds
===> Calculating affinity matrix...
   --> Time elapsed: 0.00 seconds
===> Running optimization with exaggeration=12.00, lr=200.00 for 250 iterations...
Iteration   50, KL divergence 1.0085, 50 iterations in 1.0047 sec
Iteration  100, KL divergence 0.9229, 50 iterations in 0.8066 sec
Iteration  150, KL divergence 1.0109, 50 iterations in 0.6630 sec
Iteration  200, KL divergence 0.9358, 50 iterations in 0.5697 sec
Iteration  250, KL divergence 0.9418, 50 iterations in 0.5853 sec
   --> Time elapsed: 3.63 seconds
===> Running optimization with exaggeration=1.00, lr=200.00 for 750 iterations...
Iteration   50, KL divergence 0.0159, 50 iterations in 0.5840 sec
Iteration  100, KL divergence 0.0156, 50 iterations in 0.5826 sec
Iteration  150, KL divergence 0.0156, 50 iterations in 0.5724 sec
Iteration  200, KL divergence 0.0156, 50 iterations in 0.5649 sec
Iteration 

In [5]:
from sklearn.datasets import make_spd_matrix
D = make_spd_matrix(X.shape[1])
N = make_spd_matrix(X.shape[0])

f = partial(KL_divergence_dy, X_unflattener=X_unflattener, Y_unflattener=Y_unflattener)
H = jax.jacrev(f, argnums=1)(X_flat, Y_flat)
H_pinv = np.linalg.pinv(H + 1e-3*np.eye(len(H)), hermitian=True)

f = partial(KL_divergence_dy, Y_flat=Y_flat, X_unflattener=X_unflattener, Y_unflattener=Y_unflattener)
# jvp
_, jvp_fun_lin = jax.linearize(f, X_flat)
# vjp
_, vjp_fun = vjp(f, X_flat)

time_start = time.time()
compute_cov_fun = lambda i: compute_cov_inner(vjp_fun=vjp_fun, jvp_fun_lin=jvp_fun_lin, 
                                        H_pinv_i=i, D=D, N=N, d=D.shape[0], n=N.shape[0], H_pinv=H_pinv)
final_cov = vmap(compute_cov_fun)(H_pinv)
time_end = time.time()
print('time: ', time_end - time_start)

time:  3.091120719909668


In [6]:
#print('KL divergence: ', KL_divergence(X_flat, Y_flat, X_unflattener, Y_unflattener))
#print('derivative of KL divergence w.r.t. Y: ', KL_divergence_dy(X_flat, Y_flat, X_unflattener, Y_unflattener))
#print('Hessian', H)
print('final covariance', final_cov)

final covariance [[ 2.74002017e-03  1.07968328e-04  2.07742047e-03 ... -6.90031156e-04
   1.48795341e-04 -3.85517254e-04]
 [ 1.07968015e-04  1.76459042e-04  1.84059056e-04 ... -7.54781213e-05
   1.85547942e-05 -3.62923347e-05]
 [ 2.07741861e-03  1.84059216e-04  3.85456393e-03 ... -9.64119739e-04
   1.97486457e-04 -4.85978817e-04]
 ...
 [-6.90031273e-04 -7.54781286e-05 -9.64119856e-04 ...  5.71631128e-04
  -5.76060265e-05  2.08318888e-04]
 [ 1.48795123e-04  1.85547178e-05  1.97486268e-04 ... -5.76060702e-05
   6.72587048e-05 -5.70331431e-05]
 [-3.85517458e-04 -3.62923274e-05 -4.85979253e-04 ...  2.08318947e-04
  -5.70331395e-05  3.36228113e-04]]


### Old 

In [7]:
def compute_dy_dx(f, X, Y):
  H = jax.hessian(f, argnums=1)(X_flat, Y_flat)
  #print('Hessian: ', H)
  H_pinv = np.linalg.pinv(H + 1e-3*np.eye(len(H)), hermitian=True)
  J_X_Y = jacrev(jacfwd(f, argnums=1), argnums=0)(X_flat, Y_flat)
  return np.dot(-H_pinv, J_X_Y)

f = partial(KL_divergence, X_unflattener=X_unflattener, Y_unflattener=Y_unflattener)
dy_dx = compute_dy_dx(f, X_flat, Y_flat)
final = np.dot(np.dot(dy_dx, np.kron(D, N)), dy_dx.T)
print('final cov: ', final)

final cov:  [[ 2.73999921e-03  1.07969514e-04  2.07740581e-03 ... -6.90026558e-04
   1.48795094e-04 -3.85514693e-04]
 [ 1.07969412e-04  1.76458590e-04  1.84059827e-04 ... -7.54785506e-05
   1.85548652e-05 -3.62925384e-05]
 [ 2.07740511e-03  1.84059943e-04  3.85453692e-03 ... -9.64113628e-04
   1.97485584e-04 -4.85975179e-04]
 ...
 [-6.90026616e-04 -7.54785578e-05 -9.64113569e-04 ...  5.71629032e-04
  -5.76059938e-05  2.08317710e-04]
 [ 1.48794788e-04  1.85548797e-05  1.97485613e-04 ... -5.76059938e-05
   6.72586539e-05 -5.70330158e-05]
 [-3.85514722e-04 -3.62925566e-05 -4.85975033e-04 ...  2.08317681e-04
  -5.70330521e-05  3.36226862e-04]]
